# Aim
To determine if transformer models can perform better than non-transformer models in an NLP single-label classification of an imbalanced dataset

# Task
Classify Huffington News headlines into categories

# Model types and training approaches
## (1) Huggingface's Transformer (DistilBERT)
- Using Huggingface's Pytorch Trainer API to fine-tune DistilBERT

## (2) Non-transformer models:
- Naive Bayes
- Random Forest
- Logistic Regression
- Support Vector Machine

To use k-fold cross-validation to determine which is the best performing non-transformer model

# Data treatments
## (1) None
use imbalanced dataset as is for both types of models to see if transformers have any advantages over non-transformer models for imbalanced data

## (2) Oversampling of minority classes

## (3) Undersampling of majority classes

# Metrics
Compare Accuracy and F1 scores


# (A) Transformer

In [ ]:
import pandas as pd
import numpy as np

import transformers
print(transformers.__version__)
from transformers import Trainer
